# Importing Packages

In [ ]:
# package for working with shapefiles and creating geodataframes
! pip install geopandas

     |████████████████████████████████| 1.0 MB 4.3 MB/s 
     |████████████████████████████████| 6.3 MB 31.4 MB/s 
     |████████████████████████████████| 15.4 MB 51.5 MB/s 


In [ ]:
# package for Geocodio to get latitude and longitude from location
! pip install pygeocodio

In [1]:
# install necessary packages
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import re
from geocodio import GeocodioClient
import json
import requests

pd.set_option('display.max_rows', 50)

# StartOut SPEII Data

In [2]:
# upload data collected from the dashboard
# data = pd.read_csv('/content/drive/MyDrive/Datasets/StartOut Economic Index/StartOut Economic Index.csv')
data = pd.read_csv('data/StartOut Economic Index.csv')
data.head()

,City,Region,State,CBSA,Index (Mean & All),Index (Total & All),Index (Mean & Female),Index (Total & Female),Index (Mean & LGBT),Index (Total & LGBT),...,Mean Exits (in M's),Total Funding (in B's),Total Jobs (in K's),Total Patents,Total Exits (in M's),All Entre- preneurs,Female Entre- preneurs,LGBT Entre- preneurs,GDP Per Capita (update with Census data),Population (in millions) (update with Census data)
0,Albany,Mideast,NY,"Albany-Schenectady-Troy, NY",38,40,12,34,5,3,...,4.0,0.47,4.0,44,28.0,165,18,0,65000,1.20
1,Albuquerque,Southwest,NM,"Albuquerque, NM",71,70,94,66,86,89,...,5.9,1.00,2.2,400,200.0,195,17,1,47000,1.20
2,Allentown,Mideast,PA,"Allentown-Bethlehem-Easton, PA-NJ",69,70,98,98,6,4,...,6.4,0.45,917.0,117,250.0,90,14,0,53000,0.84
3,Asheville,Southeast,NC,"Asheville, NC",48,45,31,48,5,4,...,5.9,0.38,2.4,1,0.0,72,8,0,44000,0.49
4,Atlanta,Southeast,GA,"Atlanta-Sandy Springs-Alpharetta, GA",94,91,78,60,64,11,...,7.9,22.00,120.0,1800,4500.0,2200,206,4,65000,6.60


In [3]:
# rename column names for easier referencing
data.columns = ['City', 'Region', 'State', 'NAME', 'IndexMA', 'IndexTA', 'IndexMF', 'IndexTF', 'IndexML', 'IndexTL', 
                'MFunding', 'MJobs', 'MPatents', 'MExits', 'TFunding', 'TJobs', 'TPatents', 'TExits', 'All_E', 
                'Female_E', 'LGBT_E', 'GDP_Per_Cap', 'Pop']

In [4]:
# change columns into right format
data.MFunding = data.MFunding.apply(lambda x: int(x * 10**6))
data.MExits = data.MExits.apply(lambda x: int(x * 10**6))
data.TExits = data.TExits.apply(lambda x: int(x * 10**6))
data.TFunding = data.TFunding.apply(lambda x: int(x * 10**9))
data.TJobs = data.TJobs.apply(lambda x: int(x * 10**3))
data.Pop = data.Pop.apply(lambda x: int(x * 10**6))
data['GDP_Per_Cap'] = data['GDP_Per_Cap'].astype(int)
data.TPatents = data.TPatents.astype(int)
data.All_E = data.All_E.astype(int)
data.Female_E = data.Female_E.astype(int)
data.LGBT_E = data.LGBT_E.astype(int)

In [5]:
# create column for location names to enter into Geocodio
data['Geocodio_Loc'] = data.City + '+' + data.State
data.Geocodio_Loc = data.Geocodio_Loc.apply(lambda x: re.sub(r'\s', '+', x))

In [6]:
# create column for name of main city in metro area
data['Location'] = data.City + ', ' + data.State

In [7]:
data.head()

,City,Region,State,NAME,IndexMA,IndexTA,IndexMF,IndexTF,IndexML,IndexTL,...,TJobs,TPatents,TExits,All_E,Female_E,LGBT_E,GDP_Per_Cap,Pop,Geocodio_Loc,Location
0,Albany,Mideast,NY,"Albany-Schenectady-Troy, NY",38,40,12,34,5,3,...,4000,44,28000000,165,18,0,65000,1200000,Albany+NY,"Albany, NY"
1,Albuquerque,Southwest,NM,"Albuquerque, NM",71,70,94,66,86,89,...,2200,400,200000000,195,17,1,47000,1200000,Albuquerque+NM,"Albuquerque, NM"
2,Allentown,Mideast,PA,"Allentown-Bethlehem-Easton, PA-NJ",69,70,98,98,6,4,...,917000,117,250000000,90,14,0,53000,840000,Allentown+PA,"Allentown, PA"
3,Asheville,Southeast,NC,"Asheville, NC",48,45,31,48,5,4,...,2400,1,0,72,8,0,44000,490000,Asheville+NC,"Asheville, NC"
4,Atlanta,Southeast,GA,"Atlanta-Sandy Springs-Alpharetta, GA",94,91,78,60,64,11,...,120000,1800,4500000000,2200,206,4,65000,6600000,Atlanta+GA,"Atlanta, GA"


# US Census Metro Area Shapefile

In [8]:
# upload shapefile from US Census Bureau for CBSA or Core Based Statistical Areas
# df = gpd.read_file('/content/drive/MyDrive/Datasets/StartOut Economic Index/cb_2020_us_cbsa_500k/cb_2020_us_cbsa_500k.shp')
df = gpd.read_file('data/cb_2020_us_cbsa_500k/cb_2020_us_cbsa_500k.shp')
df.head()

,CSAFP,CBSAFP,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER,geometry
0,None,11380,310M600US11380,11380,"Andrews, TX","Andrews, TX Micro Area",M2,3886850259,957039,"POLYGON ((-103.06470 32.52219, -103.00047 32.5..."
1,192,35140,310M600US35140,35140,"Newberry, SC","Newberry, SC Micro Area",M2,1632452022,44011454,"POLYGON ((-81.94372 34.20605, -81.94196 34.208..."
2,None,37540,310M600US37540,37540,"Paris, TN","Paris, TN Micro Area",M2,1455320362,81582236,"POLYGON ((-88.52940 36.17018, -88.52636 36.229..."
3,246,29900,310M600US29900,29900,"Laurinburg, NC","Laurinburg, NC Micro Area",M2,826569986,3842049,"POLYGON ((-79.69251 34.80685, -79.68822 34.809..."
4,315,35460,310M600US35460,35460,"Newport, TN","Newport, TN Micro Area",M2,1129584563,17932684,"POLYGON ((-83.31519 35.89332, -83.31078 35.895..."


In [9]:
# filter out all micropolitan statistical areas
df = df[df.LSAD == 'M1']

In [10]:
df.head()

,CSAFP,CBSAFP,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER,geometry
10,None,31740,310M600US31740,31740,"Manhattan, KS","Manhattan, KS Metro Area",M1,4752896547,137754617,"POLYGON ((-96.96346 38.99445, -96.96316 39.030..."
11,None,37900,310M600US37900,37900,"Peoria, IL","Peoria, IL Metro Area",M1,8633293349,174445163,"POLYGON ((-90.45192 40.18880, -90.45106 40.232..."
12,216,14500,310M600US14500,14500,"Boulder, CO","Boulder, CO Metro Area",M1,1881324131,36486613,"MULTIPOLYGON (((-105.08257 39.95928, -105.0768..."
13,440,41420,310M600US41420,41420,"Salem, OR","Salem, OR Metro Area",M1,4978011395,34703808,"POLYGON ((-123.72466 45.04443, -123.72437 45.0..."
14,None,25940,310M600US25940,25940,"Hilton Head Island-Bluffton, SC","Hilton Head Island-Bluffton, SC Metro Area",M1,3188786450,1020102631,"POLYGON ((-81.28130 32.55644, -81.22258 32.603..."


# Merging Datasets

In [11]:
# merge shapefile with website datafrane
geo_df = df.merge(data, on='NAME', how='right')
geo_df.head()

,CSAFP,CBSAFP,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER,geometry,...,TJobs,TPatents,TExits,All_E,Female_E,LGBT_E,GDP_Per_Cap,Pop,Geocodio_Loc,Location
0,104,10580,310M600US10580,10580,"Albany-Schenectady-Troy, NY","Albany-Schenectady-Troy, NY Metro Area",M1,7282379774,172340496,"POLYGON ((-74.71158 42.51780, -74.69844 42.533...",...,4000,44,28000000,165,18,0,65000,1200000,Albany+NY,"Albany, NY"
1,106,10740,310M600US10740,10740,"Albuquerque, NM","Albuquerque, NM Metro Area",M1,24043885295,31811673,"MULTIPOLYGON (((-106.27136 35.85557, -106.2483...",...,2200,400,200000000,195,17,1,47000,1200000,Albuquerque+NM,"Albuquerque, NM"
2,None,10900,310M600US10900,10900,"Allentown-Bethlehem-Easton, PA-NJ","Allentown-Bethlehem-Easton, PA-NJ Metro Area",M1,3763211244,58185339,"POLYGON ((-75.99764 40.91326, -75.98774 40.916...",...,917000,117,250000000,90,14,0,53000,840000,Allentown+PA,"Allentown, PA"
3,120,11700,310M600US11700,11700,"Asheville, NC","Asheville, NC Metro Area",M1,5264465452,21752837,"POLYGON ((-83.25611 35.70396, -83.25511 35.707...",...,2400,1,0,72,8,0,44000,490000,Asheville+NC,"Asheville, NC"
4,122,12060,310M600US12060,12060,"Atlanta-Sandy Springs-Alpharetta, GA","Atlanta-Sandy Springs-Alpharetta, GA Metro Area",M1,22495780629,386874693,"POLYGON ((-85.38647 33.90172, -85.31257 33.900...",...,120000,1800,4500000000,2200,206,4,65000,6600000,Atlanta+GA,"Atlanta, GA"


In [12]:
len(geo_df)

99

In [13]:
geo_df.geometry.isna().sum()

0

# Obtaining Latitude and Longitude of Metro Areas

In [14]:
client = GeocodioClient('158110b61505b87a885819570891115766b9891')

In [15]:
url = 'https://api.geocod.io/v1.7/geocode?q=New+York+NY&api_key=158110b61505b87a885819570891115766b9891&format=simple'
response = requests.get(url)
location = json.loads(response.content.decode('utf-8'))

In [18]:
location

{'address': 'New York, NY 10001',
 'lat': 40.750422,
 'lng': -73.996328,
 'accuracy': 1,
 'accuracy_type': 'place',
 'source': 'TIGER/Line® dataset from the US Census Bureau'}

In [16]:
geo_list = list(geo_df.Geocodio_Loc.values)

In [17]:
loc_list = []

for city in geo_list:
    url = 'https://api.geocod.io/v1.7/geocode?q=' + city + '&api_key=158110b61505b87a885819570891115766b9891&format=simple'
    response = requests.get(url)
    location = json.loads(response.content.decode('utf-8'))
    loc_list.append(location)

In [18]:
loc_list

[{'address': 'Albany, NY 12201',
  'lat': 42.614852,
  'lng': -73.970812,
  'accuracy': 1,
  'accuracy_type': 'place',
  'source': 'TIGER/Line® dataset from the US Census Bureau'},
 {'address': 'Albuquerque, NM 87101',
  'lat': 35.199592,
  'lng': -106.644831,
  'accuracy': 1,
  'accuracy_type': 'place',
  'source': 'TIGER/Line® dataset from the US Census Bureau'},
 {'address': 'Allentown, PA 18101',
  'lat': 40.602658,
  'lng': -75.469236,
  'accuracy': 1,
  'accuracy_type': 'place',
  'source': 'TIGER/Line® dataset from the US Census Bureau'},
 {'address': 'Asheville, NC 28801',
  'lat': 35.602711,
  'lng': -82.567281,
  'accuracy': 1,
  'accuracy_type': 'place',
  'source': 'TIGER/Line® dataset from the US Census Bureau'},
 {'address': 'Atlanta, GA 30301',
  'lat': 33.844371,
  'lng': -84.47405,
  'accuracy': 1,
  'accuracy_type': 'place',
  'source': 'TIGER/Line® dataset from the US Census Bureau'},
 {'address': 'Austin, TX 78701',
  'lat': 30.268335,
  'lng': -97.741382,
  'accura

In [19]:
lat_list = []
long_list = []

for i in range(len(loc_list)):
    lat_list.append(loc_list[i]['lat'])
    
for i in range(len(loc_list)):
    long_list.append(loc_list[i]['lng'])


In [20]:
len(lat_list)

99

In [21]:
len(long_list)

99

In [22]:
geo_df['Lat'] = lat_list
geo_df['Long'] = long_list
geo_df.head()

,CSAFP,CBSAFP,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER,geometry,...,TExits,All_E,Female_E,LGBT_E,GDP_Per_Cap,Pop,Geocodio_Loc,Location,Lat,Long
0,104,10580,310M600US10580,10580,"Albany-Schenectady-Troy, NY","Albany-Schenectady-Troy, NY Metro Area",M1,7282379774,172340496,"POLYGON ((-74.71158 42.51780, -74.69844 42.533...",...,28000000,165,18,0,65000,1200000,Albany+NY,"Albany, NY",42.614852,-73.970812
1,106,10740,310M600US10740,10740,"Albuquerque, NM","Albuquerque, NM Metro Area",M1,24043885295,31811673,"MULTIPOLYGON (((-106.27136 35.85557, -106.2483...",...,200000000,195,17,1,47000,1200000,Albuquerque+NM,"Albuquerque, NM",35.199592,-106.644831
2,None,10900,310M600US10900,10900,"Allentown-Bethlehem-Easton, PA-NJ","Allentown-Bethlehem-Easton, PA-NJ Metro Area",M1,3763211244,58185339,"POLYGON ((-75.99764 40.91326, -75.98774 40.916...",...,250000000,90,14,0,53000,840000,Allentown+PA,"Allentown, PA",40.602658,-75.469236
3,120,11700,310M600US11700,11700,"Asheville, NC","Asheville, NC Metro Area",M1,5264465452,21752837,"POLYGON ((-83.25611 35.70396, -83.25511 35.707...",...,0,72,8,0,44000,490000,Asheville+NC,"Asheville, NC",35.602711,-82.567281
4,122,12060,310M600US12060,12060,"Atlanta-Sandy Springs-Alpharetta, GA","Atlanta-Sandy Springs-Alpharetta, GA Metro Area",M1,22495780629,386874693,"POLYGON ((-85.38647 33.90172, -85.31257 33.900...",...,4500000000,2200,206,4,65000,6600000,Atlanta+GA,"Atlanta, GA",33.844371,-84.474050


In [23]:
geo_df_2 = geo_df.copy()

In [24]:
geo_df.drop(columns=['CSAFP', 'CBSAFP', 'AFFGEOID', 'NAMELSAD', 'LSAD', 'ALAND', 'AWATER', 'Geocodio_Loc'], inplace=True)
geo_df.head()

,GEOID,NAME,geometry,City,Region,State,IndexMA,IndexTA,IndexMF,IndexTF,...,TPatents,TExits,All_E,Female_E,LGBT_E,GDP_Per_Cap,Pop,Location,Lat,Long
0,10580,"Albany-Schenectady-Troy, NY","POLYGON ((-74.71158 42.51780, -74.69844 42.533...",Albany,Mideast,NY,38,40,12,34,...,44,28000000,165,18,0,65000,1200000,"Albany, NY",42.614852,-73.970812
1,10740,"Albuquerque, NM","MULTIPOLYGON (((-106.27136 35.85557, -106.2483...",Albuquerque,Southwest,NM,71,70,94,66,...,400,200000000,195,17,1,47000,1200000,"Albuquerque, NM",35.199592,-106.644831
2,10900,"Allentown-Bethlehem-Easton, PA-NJ","POLYGON ((-75.99764 40.91326, -75.98774 40.916...",Allentown,Mideast,PA,69,70,98,98,...,117,250000000,90,14,0,53000,840000,"Allentown, PA",40.602658,-75.469236
3,11700,"Asheville, NC","POLYGON ((-83.25611 35.70396, -83.25511 35.707...",Asheville,Southeast,NC,48,45,31,48,...,1,0,72,8,0,44000,490000,"Asheville, NC",35.602711,-82.567281
4,12060,"Atlanta-Sandy Springs-Alpharetta, GA","POLYGON ((-85.38647 33.90172, -85.31257 33.900...",Atlanta,Southeast,GA,94,91,78,60,...,1800,4500000000,2200,206,4,65000,6600000,"Atlanta, GA",33.844371,-84.474050


In [25]:
geo_df[geo_df.State == 'WI']

,GEOID,NAME,geometry,City,Region,State,IndexMA,IndexTA,IndexMF,IndexTF,...,TPatents,TExits,All_E,Female_E,LGBT_E,GDP_Per_Cap,Pop,Location,Lat,Long
56,31540,"Madison, WI","POLYGON ((-90.42991 43.11841, -90.42954 43.179...",Madison,Great Lakes,WI,76,79,65,48,...,530,66000000,521,42,0,72000,890000,"Madison, WI",43.060926,-88.230636
59,33340,"Milwaukee-Waukesha, WI","POLYGON ((-88.54215 42.90499, -88.54206 42.927...",Milwaukee,Great Lakes,WI,87,86,85,92,...,187,78000000,192,31,0,64000,2000000,"Milwaukee, WI",43.043213,-87.894734


In [27]:
geo_df.loc[geo_df.NAME == 'Madison, WI', 'Lat'] = 43.0731
geo_df.loc[geo_df.NAME == 'Madison, WI', 'Long'] = -89.4012
geo_df.loc[geo_df.City == 'Milwaukee', 'Lat'] = 43.0389
geo_df.loc[geo_df.City == 'Milwaukee', 'Long'] = -87.9065

In [28]:
type(geo_df)

geopandas.geodataframe.GeoDataFrame

In [29]:
geo_df.to_file('geo_df.geojson', driver='GeoJSON')

In [30]:
geo_df_2.columns

Index(['CSAFP', 'CBSAFP', 'AFFGEOID', 'GEOID', 'NAME', 'NAMELSAD', 'LSAD',
       'ALAND', 'AWATER', 'geometry', 'City', 'Region', 'State', 'IndexMA',
       'IndexTA', 'IndexMF', 'IndexTF', 'IndexML', 'IndexTL', 'MFunding',
       'MJobs', 'MPatents', 'MExits', 'TFunding', 'TJobs', 'TPatents',
       'TExits', 'All_E', 'Female_E', 'LGBT_E', 'GDP_Per_Cap', 'Pop',
       'Geocodio_Loc', 'Location', 'Lat', 'Long'],
      dtype='object')

In [31]:
geo_df_2.loc[geo_df_2.NAME == 'Madison, WI', 'Lat'] = 43.0731
geo_df_2.loc[geo_df_2.NAME == 'Madison, WI', 'Long'] = -89.4012
geo_df_2.loc[geo_df_2.City == 'Milwaukee', 'Lat'] = 43.0389
geo_df_2.loc[geo_df_2.City == 'Milwaukee', 'Long'] = -87.9065

In [32]:
geo_df_2.drop(columns=['CSAFP', 'CBSAFP', 'AFFGEOID', 'NAMELSAD', 'LSAD', 'ALAND', 'AWATER', 'Geocodio_Loc',
                       'IndexMA', 'IndexTA', 'IndexMF', 'IndexTF', 'IndexML', 'IndexTL', 'MFunding',
                       'MJobs', 'MPatents', 'MExits', 'TFunding', 'TJobs', 'TPatents', 'TExits', 
                       'All_E', 'Female_E', 'LGBT_E',], inplace=True)
geo_df_2.head()

,GEOID,NAME,geometry,City,Region,State,GDP_Per_Cap,Pop,Location,Lat,Long
0,10580,"Albany-Schenectady-Troy, NY","POLYGON ((-74.71158 42.51780, -74.69844 42.533...",Albany,Mideast,NY,65000,1200000,"Albany, NY",42.614852,-73.970812
1,10740,"Albuquerque, NM","MULTIPOLYGON (((-106.27136 35.85557, -106.2483...",Albuquerque,Southwest,NM,47000,1200000,"Albuquerque, NM",35.199592,-106.644831
2,10900,"Allentown-Bethlehem-Easton, PA-NJ","POLYGON ((-75.99764 40.91326, -75.98774 40.916...",Allentown,Mideast,PA,53000,840000,"Allentown, PA",40.602658,-75.469236
3,11700,"Asheville, NC","POLYGON ((-83.25611 35.70396, -83.25511 35.707...",Asheville,Southeast,NC,44000,490000,"Asheville, NC",35.602711,-82.567281
4,12060,"Atlanta-Sandy Springs-Alpharetta, GA","POLYGON ((-85.38647 33.90172, -85.31257 33.900...",Atlanta,Southeast,GA,65000,6600000,"Atlanta, GA",33.844371,-84.474050


In [33]:
geo_df_2.isna().sum()

GEOID          0
NAME           0
geometry       0
City           0
Region         0
State          0
GDP_Per_Cap    0
Pop            0
Location       0
Lat            0
Long           0
dtype: int64

In [79]:
geo_df_2[geo_df_2.GEOID.isna()]

,GEOID,NAME,geometry,City,Region,State,GDP_Per_Cap,Pop,Location,Lat,Long
48,NaN,"Lancaster,PA",None,Lancaster,Mideast,PA,54000,540000,"Lancaster, PA",40.076553,-76.31068


In [34]:
geo_df_2.to_file('geo_df_2.geojson', driver='GeoJSON')

In [35]:
list(geo_df_2.Location.values)

['Albany, NY',
 'Albuquerque, NM',
 'Allentown, PA',
 'Asheville, NC',
 'Atlanta, GA',
 'Austin, TX',
 'Baltimore, MD',
 'Bend, OR',
 'Birmingham, AL',
 'Bloomington, IN',
 'Boise City, ID',
 'Boston, MA',
 'Buffalo, NY',
 'Burlington, VT',
 'Cape Coral, FL',
 'Cedar Rapids, IA',
 'Champaign, IL',
 'Charleston, SC',
 'Charlotte, NC',
 'Charlottesville, VA',
 'Chattanooga, TN',
 'Chicago, IL',
 'Cincinnati, OH',
 'Cleveland, OH',
 'Colorado Springs, CO',
 'Columbia, SC',
 'Columbus, OH',
 'Dallas, TX',
 'Denver, CO',
 'Des Moines, IA',
 'Detroit, MI',
 'Fayetteville, AR',
 'Fort Collins, CO',
 'Gainesville, FL',
 'Grand Rapids, MI',
 'Greensboro, NC',
 'Greenville, SC',
 'Hartford, CT',
 'Honolulu, HI',
 'Houston, TX',
 'Huntsville, AL',
 'Indianapolis, IN',
 'Ithaca, NY',
 'Jacksonville, FL',
 'Kalamazoo, MI',
 'Kansas City, MO',
 'Knoxville, TN',
 'Lafayette, IN',
 'Lancaster, PA',
 'Lansing, MI',
 'Las Vegas, NV',
 'Lexington, KY',
 'Lincoln, NE',
 'Little Rock, AR',
 'Los Angeles, C